In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import torch 
import torch.nn.functional as F 

from tqdm.notebook import tqdm
from model import *

# Load MNIST Handwritten Dataset

In [20]:
from torchvision import datasets 
from torchvision import transforms as T

In [21]:
train_augs = T.Compose([
    T.ToTensor(),
    T.Normalize(mean = 0.5, std = 0.5)
])

valid_augs = T.Compose([
    T.ToTensor(), 
    T.Normalize(mean = 0.5, std = 0.5)
])

In [22]:
trainset = datasets.MNIST('./', download = False, train = True, transform = train_augs)
testset = datasets.MNIST('./', download = False, train = False, transform = valid_augs)

In [23]:
trainset, validset = torch.utils.data.random_split(trainset, [50000, 10000])

In [24]:
print(f"Size of trainset : {len(trainset)}")
print(f"Size of validset : {len(validset)}")
print(f"Size of testset : {len(testset)}")

Size of trainset : 50000
Size of validset : 10000
Size of testset : 10000


# Load Dataset in Batches

In [25]:
from torch.utils.data import DataLoader

In [26]:
bs = 64

trainloader = DataLoader(trainset, batch_size = bs, shuffle = True)
validloader = DataLoader(validset, batch_size = bs)
testloader = DataLoader(testset, batch_size = bs)

In [27]:
print(f'Total no. of batches in trainloader : {len(trainloader)}')
print(f'Total no. of batches in validloader : {len(validloader)}')
print(f'Total no. of batches in testloader : {len(testloader)}')

Total no. of batches in trainloader : 782
Total no. of batches in validloader : 157
Total no. of batches in testloader : 157


# Load Model

In [28]:
from model import DigitModel

model = DigitModel()
model.to('cpu')

DigitModel(
  (cnn_block): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (linear_block): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1568, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_featu

# Train and Eval Function

In [29]:

def multiclass_accuracy(y_pred,y_true):
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))


In [30]:
def train_fn(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    
    for images, labels in tqdm(dataloader):
        images = images.to('cpu')
        labels = labels.to('cpu')
        
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits, labels)
    return total_loss / len(dataloader), total_acc / len(dataloader)

In [31]:
def eval_fn(model, dataloader, criterion):
    
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    
    with torch.no_grad():

        for images, labels in tqdm(dataloader):
            images = images.to('cpu')
            labels = labels.to('cpu')

            logits = model(images)
            loss = criterion(logits, labels)

            total_loss += loss.item()
            total_acc += multiclass_accuracy(logits, labels)

        return total_loss / len(dataloader), total_acc / len(dataloader)
    

# Training Loop

In [32]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.003)

In [17]:
best_valid_loss = np.Inf

for i in range(20):
    
    train_loss, train_acc = train_fn(model, trainloader, criterion, optimizer)
    valid_loss, valid_acc = eval_fn(model, trainloader, criterion)
    
    print(f"Epoch {i+1} Train Loss : {train_loss} Train ACC : {train_acc}")
    print(f"Epoch {i+1} Valid Loss : {valid_loss} Valid ACC : {valid_acc}")
          
    if valid_loss < best_valid_loss:
          torch.save(model.state_dict(), 'best_weights.pt')
          print('SAVED BEST MODEL')
          best_valid_loss = valid_loss

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1 Train Loss : 0.26353897907725915 Train ACC : 0.920516312122345
Epoch 1 Valid Loss : 0.06906349812504774 Valid ACC : 0.9808783531188965
SAVED BEST MODEL


  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

...

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 19 Train Loss : 0.03289883758234259 Train ACC : 0.9921875
Epoch 19 Valid Loss : 0.008095414300101038 Valid ACC : 0.9976022839546204


  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 20 Train Loss : 0.028606828339304172 Train ACC : 0.9930266737937927
Epoch 20 Valid Loss : 0.006959255022450112 Valid ACC : 0.9982616901397705
